In [24]:
import pandas as pd
import numpy as np

In [25]:
train_df = pd.read_csv('D:\\Competitions\\DataStorm-2022\\semi-final\\data\\train_data.csv')
val_df = pd.read_csv('D:\\Competitions\\DataStorm-2022\\semi-final\\data\\validation_data.csv')
test_df = pd.read_csv('D:\\Competitions\\DataStorm-2022\\semi-final\\data\\test_data.csv')

In [26]:
train_df.head()

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,1071115,12/19/2021,1
1,category_4,210652,2/13/2022,1
2,category_2,877624,12/19/2021,1
3,category_1,371104,12/19/2021,3
4,category_1,1090294,12/19/2021,3


In [27]:
train_df.shape

(20651, 4)

In [28]:
train_udf = train_df.copy().iloc[:, 1:4].sort_values(by=['ItemCode', 'DateID'])
train_udf.head()

,ItemCode,DateID,DailySales
8316,3418,1/1/2022,4
14144,3418,1/10/2022,7
15931,3418,1/11/2022,7
13319,3418,1/12/2022,14
1772,3418,1/13/2022,28


In [29]:
start_date = min(train_udf.DateID)
end_date = max(train_udf.DateID)
# print(start_date, end_date)
def df_func(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    _x['DateID'] = pd.to_datetime(_x['DateID'])
    _x = _x.set_index('DateID')
    __x = _x.resample('D').sum().fillna(0)
    # print(__x)
    return __x

In [30]:
res_df = train_udf.groupby('ItemCode').apply(lambda x: df_func(x)).reset_index().sort_values(by=['ItemCode', 'DateID'])
# train_udf.groupby('ItemCode').apply(lambda x: print(x.index)).reset_index()
res_df.head()

,ItemCode,DateID,DailySales
0,3418,2021-10-01,18
1,3418,2021-10-02,3
2,3418,2021-10-03,8
3,3418,2021-10-04,2
4,3418,2021-10-05,4


In [31]:
I2C_map = pd.Series(train_df.CategoryCode.values, index=train_df.ItemCode).to_dict()
res_df['CategoryCode'] = res_df['ItemCode'].map(I2C_map)

In [32]:
res_df.columns = ['ItemCode', 'Date', 'DailySales', 'CategoryCode']
res_df = res_df.sort_values(by=['ItemCode', 'Date'])
res_df.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [33]:
res_df.to_csv('../data/train_dataV6.csv')

---
Make list

In [34]:
_df1 = res_df.copy().loc[:, ['ItemCode', 'DailySales']].groupby('ItemCode').agg(list).reset_index()
_df1.head()

,ItemCode,DailySales
0,3418,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,3427,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,7666,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,9925,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,16936,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."


In [35]:
_df1['CategoryCode'] = res_df['ItemCode'].map(I2C_map)
_df1.head()

,ItemCode,DailySales,CategoryCode
0,3418,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ...",category_1
1,3427,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,...",category_1
2,7666,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,...",category_1
3,9925,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ...",category_1
4,16936,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,...",category_1


In [36]:
_df1.to_csv('../data/DSalesPointsV2.csv', index=False)

***

In [37]:
train_items = set(train_df['ItemCode'])
val_items = set(val_df['ItemCode'])
test_items = set(test_df['ItemCode'])

In [38]:
stage_map = {i: 'validation' for i in val_items}
stage_map.update({i: 'test' for i in test_items})
stage_map.update({836584: 'train-only', 1090249: 'train-only'})

In [39]:
df2 = res_df.copy()
df2.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [40]:
sales_point = df2.groupby('ItemCode')['DailySales'].count().reset_index()
sales = df2.groupby('ItemCode')['DailySales'].agg(list)
sales_point['stage'] = sales_point['ItemCode'].map(stage_map)
sales_point['sales'] = sales_point['ItemCode'].map(sales)
sales_point.columns = ['ItemCode', '#sales', 'stage', 'DailySales']
sales_point.head()
# sales_point1.to_csv('../data/SalesPointsV1.csv')

,ItemCode,#sales,stage,DailySales
0,3418,140,validation,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,3427,138,test,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,7666,140,test,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,9925,138,test,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,16936,140,validation,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."


In [41]:
sales_point.to_csv('../data/DSalesPointsV2.csv', index=False)

***

In [42]:
df2['WoY'] = df2["Date"].dt.weekofyear
df2.head()

C:\Users\ACER\AppData\Local\Temp\ipykernel_17448\2016220253.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df2['WoY'] = df2["Date"].dt.weekofyear


,ItemCode,Date,DailySales,CategoryCode,WoY
0,3418,2021-10-01,18,category_1,39
1,3418,2021-10-02,3,category_1,39
2,3418,2021-10-03,8,category_1,39
3,3418,2021-10-04,2,category_1,40
4,3418,2021-10-05,4,category_1,40


In [43]:
df3 = df2.loc[:, ['ItemCode', 'DailySales', 'WoY']]
df3.head()

,ItemCode,DailySales,WoY
0,3418,18,39
1,3418,3,39
2,3418,8,39
3,3418,2,40
4,3418,4,40


In [44]:
def df_func1(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    print(_x)
    __x = _x.groupby('WoY').sum().reset_index()
    print(__x)
    return __x

In [45]:
pd.set_option('display.max_rows', None)
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

     DailySales  WoY
0            18   39
1             3   39
2             8   39
3             2   40
4             4   40
5             2   40
6             8   40
7            12   40
8             0   40
9            14   40
10            2   41
11            9   41
12            8   41
13            5   41
14           15   41
15            2   41
16            0   41
17           14   42
18            4   42
19           12   42
20            2   42
21            1   42
22            6   42
23            2   42
24           13   43
25            5   43
26            5   43
27            6   43
28           10   43
29            2   43
30            3   43
31            8   44
32           12   44
33            8   44
34           10   44
35            7   44
36            1   44
37            0   44
38           14   45
39            3   45
40            6   45
41            7   45
42            3   45
43            5   45
44            5   45
45            7   46
46           

WoY  DailySales
ItemCode                   
3418     0    1          83
         1    2          85
         2    3          89
         3    4          67
         4    5         109

In [46]:
df5 = df3.groupby(['ItemCode', 'WoY']).sum().reset_index()
df5.head()
# df5.to_csv('../data/train_dataV5.csv')

,ItemCode,WoY,DailySales
0,3418,1,83
1,3418,2,85
2,3418,3,89
3,3418,4,67
4,3418,5,109


In [47]:
I2C_map = pd.Series(train_df.CategoryCode.values,index=train_df.ItemCode).to_dict()
df5['CategoryCode'] = df5['ItemCode'].map(I2C_map)
df5.columns = ['ItemCode', 'WoY', 'WeeklySales', 'CategoryCode']
df5.head()

,ItemCode,WoY,WeeklySales,CategoryCode
0,3418,1,83,category_1
1,3418,2,85,category_1
2,3418,3,89,category_1
3,3418,4,67,category_1
4,3418,5,109,category_1


In [48]:
df6 = df5.copy().loc[:, ['CategoryCode', 'ItemCode', 'WoY', 'WeeklySales']]
df6.head()

,CategoryCode,ItemCode,WoY,WeeklySales
0,category_1,3418,1,83
1,category_1,3418,2,85
2,category_1,3418,3,89
3,category_1,3418,4,67
4,category_1,3418,5,109


In [49]:
df6.to_csv('../data/sample_train1.csv')

In [50]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4083 entries, 0 to 4082
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ItemCode      4083 non-null   int64 
 1   WoY           4083 non-null   int64 
 2   WeeklySales   4083 non-null   int64 
 3   CategoryCode  4083 non-null   object
dtypes: int64(3), object(1)
memory usage: 127.7+ KB


In [51]:
df5['WoY'].unique()

array([ 1,  2,  3,  4,  5,  6,  7, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52], dtype=int64)

In [52]:
woy_map = {n: i for i,n in enumerate(range(39, 53), 1)}
woy_map.update({n: i for i,n in enumerate(range(1, 7), 15)})
woy_map

{39: 1,
 40: 2,
 41: 3,
 42: 4,
 43: 5,
 44: 6,
 45: 7,
 46: 8,
 47: 9,
 48: 10,
 49: 11,
 50: 12,
 51: 13,
 52: 14,
 1: 15,
 2: 16,
 3: 17,
 4: 18,
 5: 19,
 6: 20}

In [53]:
df5['WoY'] = df5['WoY'].map(woy_map)
df5.head()

,ItemCode,WoY,WeeklySales,CategoryCode
0,3418,15.0,83,category_1
1,3418,16.0,85,category_1
2,3418,17.0,89,category_1
3,3418,18.0,67,category_1
4,3418,19.0,109,category_1


In [54]:
sales_point1 = df5.groupby('ItemCode')['WeeklySales'].count().reset_index()
sales = df5.groupby('ItemCode')['WeeklySales'].agg(list)
sales_point1['stage'] = sales_point1['ItemCode'].map(stage_map)
sales_point1['sales'] = sales_point1['ItemCode'].map(sales)
sales_point1.columns = ['ItemCode', '#sales', 'stage', 'WeeklySales']
sales_point1.head()
sales_point1.to_csv('../data/WSalesPointsV2.csv')

In [55]:
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

     DailySales  WoY
0            18   39
1             3   39
2             8   39
3             2   40
4             4   40
5             2   40
6             8   40
7            12   40
8             0   40
9            14   40
10            2   41
11            9   41
12            8   41
13            5   41
14           15   41
15            2   41
16            0   41
17           14   42
18            4   42
19           12   42
20            2   42
21            1   42
22            6   42
23            2   42
24           13   43
25            5   43
26            5   43
27            6   43
28           10   43
29            2   43
30            3   43
31            8   44
32           12   44
33            8   44
34           10   44
35            7   44
36            1   44
37            0   44
38           14   45
39            3   45
40            6   45
41            7   45
42            3   45
43            5   45
44            5   45
45            7   46
46           

WoY  DailySales
ItemCode                   
3418     0    1          83
         1    2          85
         2    3          89
         3    4          67
         4    5         109

In [56]:
df4.to_csv('../data/train_dataV4.csv')

In [57]:
val_df = pd.read_csv('../data/validation_data.csv')
test_df = pd.read_csv('../data/test_data.csv')

In [58]:
val_df.head()

,CategoryCode,ItemCode,Week,OnPromo,WeeklySales
0,category_2,1006090,w1,0,9
1,category_2,1105009,w1,0,15
2,category_2,1098502,w3,0,7
3,category_1,1081321,w3,0,25
4,category_4,1074823,w1,0,24


In [59]:
week_map = {
    'w1': 21,
    'w2': 22,
    'w3': 23,
    'w4': 24
}

In [60]:
val_df['Week'] = val_df['Week'].map(week_map)
test_df['Week'] = test_df['Week'].map(week_map)

val_df.head()

,CategoryCode,ItemCode,Week,OnPromo,WeeklySales
0,category_2,1006090,21,0,9
1,category_2,1105009,21,0,15
2,category_2,1098502,23,0,7
3,category_1,1081321,23,0,25
4,category_4,1074823,21,0,24


In [61]:
test_df.head()

,CategoryCode,ItemCode,Week,OnPromo,PredictedSales
0,category_1,1048975,24,0,NaN
1,category_1,43738,24,0,NaN
2,category_2,1044502,21,0,NaN
3,category_2,913561,24,0,NaN
4,category_2,216151,24,0,NaN


In [63]:
val_df.columns = ['CategoryCode', 'ItemCode', 'WoY', "OnPromo", 'WeeklySales']
test_df.columns = ['CategoryCode', 'ItemCode', 'WoY', "OnPromo", 'WeeklySales']

In [64]:
val_df.to_csv('../data/sample_val1.csv')
test_df.to_csv('../data/sample_test1.csv')